# Program Overview



# Headers

In [1]:
import header
paths = header.setup_environment()
for k,v in paths.items():
    print(k, '->', v)
    

Darwin
HOME -> /Users/thompsong
Developer -> /Users/thompsong/Developer
repodir -> /Users/thompsong/Developer/KSCRocketSeismoHydrology/Python/new_workflow
work -> /Users/thompsong/work
local_outdir -> /Users/thompsong/work/PROJECTS/KSC_EROSION
DROPBOX_TOP -> /Users/thompsong/Dropbox
new_data -> /Users/thompsong/data/KSCwell
DROPBOX_DATA_TOP -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA
dropbox_outdir -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/new_workflow
WELLDATA_TOP -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData
TOB3_DIR -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/Uploads
transducersCSVfile -> /Users/thompsong/Developer/KSCRocketSeismoHydrology/Python/new_work

In [2]:
import os
import glob
import pandas as pd


/var/folders/4q/5426zn1d72vgcb6h658n_8800000gn/T/ipykernel_67655/976997968.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


'\nimport obspy\nfrom IPython.display import clear_output\nFILELOOKUPCSV = os.path.join(paths[\'new_data\'], \'reverse_lookup.csv\')\nfilesnotprocessedfile = os.path.join(paths[\'new_data\'], \'files_not_processed.txt\')\nMASTERCSV = os.path.join(paths[\'new_data\'], f\'data_inventory.csv\')\nerase=False\nif erase:\n    os.system(f\'rm -rf {paths[\'new_data\']}/*\')\nif not os.path.isfile(FILELOOKUPCSV):\n    os.system(f\'echo "fullpath, outpklfullpath, basename, starttime, endtime, seqno" > {FILELOOKUPCSV}\')\nlookupdf = pd.read_csv(FILELOOKUPCSV)\nif os.path.isfile(MASTERCSV):\n    allmasterrows = pd.read_csv(MASTERCSV).to_dict(\'records\')\nelse:\n    allmasterrows=[]\n    \nimport libWellData as LLE\ntransducersDF = pd.read_csv(paths[\'transducersCSVfile\'])\n'

# Functions

In [3]:
def process_tob3file(tob3file):
    print(tob3file)
    startdatetime=None
    with open(tob3file, 'rb') as f:
        firstline = f.readline().decode()
    #print(firstline)
    fields = firstline.split(',')
    for field in fields:
        if len(field)>5 and field[0:5]=='\"2022':
            startdatetime = field[1:-2].replace('\"', '')
    thisdict = {'file':os.path.basename(tob3file), 'datetime':startdatetime}
    return thisdict


'\ndef copy_to_raw(df, pklfile, to_csv=True):\n    print(\'- copying data to %s\' % pklfile)       \n    if to_csv:       \n        df.to_csv(pklfile.replace(\'.pkl\', \'.csv\'), index=False)\n    else:\n        df.to_pickle(pklfile)\n\ndef write_masterdf(allmasterrows):\n    if len(allmasterrows)>0:\n        masterdf = pd.DataFrame(allmasterrows)\n        print(f\'Writing/updating {MASTERCSV}\')\n        if \'starttime\' in masterdf.columns:\n            masterdf.sort_values(by=[\'starttime\',\'uploadfolder\',\'sampratefolder\',\'samprate\']).to_csv(MASTERCSV, index=False)\n        else:\n            masterdf.to_csv(MASTERCSV, index=False)\n\ndef process_file(dirpath, file, filenum, load=False):\n    clear_output()\n    print(f\'Processing file {filenum}: {file} in {dirpath}\')\n    fparts = file.split(\'.\')\n    if len(fparts)==4:\n        uploadfolder, sampratefolder, basefilename, ext = fparts\n    elif len(fparts)==2:\n        uploadfolder = \'unknown\'\n        sampratefolder = 

# Main program

In [ ]:
lod = []
tob3dir = paths['TOB3_DIR']
for uploaddir in sorted(glob.glob(os.path.join(tob3dir, '202*'))):
    for dirorfile in sorted(glob.glob(os.path.join(uploaddir, '*'))):
        if os.path.isdir(dirorfile):
            sampratedir = dirorfile
            for tob3file in sorted(glob.glob(os.path.join(sampratedir, '*.dat'))):
                thisdict = process_tob3file(tob3file)
                thisdict['uploaddir']=os.path.basename(uploaddir)
                thisdict['sampratedir']=os.path.basename(sampratedir)
                lod.append(thisdict)

        elif dirorfile[-4:] == '.dat':
            sampratedir = ''
            tob3file = dirorfile
            thisdict = process_tob3file(tob3file)
            thisdict['uploaddir']=os.path.basename(uploaddir)
            thisdict['sampratedir']=os.path.basename(sampratedir)
            lod.append(thisdict)
  

In [5]:
df = pd.DataFrame(lod)
df.sort_values(by='datetime', inplace=True)
df.to_csv('list_of_tob3_files_new.csv', index=False)